# MODEL COMPRESSION

**Motivation**: Design and implementation of object detection model that could run efficiently on edge devices

###  Challenges of Deep learning on edge devices
+ Model size
+ Inference speed (number of computations)


| Operation | Energy[pJ] | Relative cost |
| --- | --- | --- |
| 32bit int ADD | 0.1 | 1 |
| 32bit float ADD | 0.9 | 9 |
| 32bit register file | 1 | 10 |
| 32bit int MULT | 3.1 | 31 |
| 32bit float MULT | 3.7 | 37 |
| 32bit SRAM Cash | 5 | 50 |
| <font color=red>32bit DRAM Memory</font> | <font color=red>640</font> | <font color=red>6400</font> |

## Model compression pipeline
The most problem of runnig deep models on edge devices is due to the memory size and the number of computations required at inference.


+ **Pruning** would help to remove redundancy and sparsity in the model weights
+ **Quantization** would help in storage as well as computation as the numbers will be represented by few bits
+ I am suggesting to add **Tensor decomposition**: the idea is to compute the rank of the weight matrix and store it more efficiently

<img src="tensor_decomposition2.jpg"
     alt="Markdown Monster icon"
     style="float: left; margin-right: 10px;" />

## Tensor decomposition

Tensor decomposition could help to do model compression.
The idea behind is to compute the rank of the weight matrix and store the parameters more efficiently.

#### Example of matrix decomposition for reducing the number of parameters
Assume $A = \begin{bmatrix} 3 & 2 & 2 \\ 2 & 3 & -2 \end{bmatrix}$
 is a rectangular matrix, $A \in I\!R^{2x3}$
 + if we assume this matrix to be our weight matrix, the total number of parameters will be <font color=red>$2 \times 3 = 6$</font> 
 + Using SVD A can be represented as $A=U \Sigma V^T$
 
 The SVD of matrix $A$ will be: $A = 
\begin{bmatrix} \frac{1}{\sqrt{2}} & \frac{1}{\sqrt{2}}  \\ \frac{1}{\sqrt{2}} & -\frac{1}{\sqrt{2}} \end{bmatrix}
\begin{bmatrix} 5 & 0 & 0 \\ 0 & 3 & 0 \end{bmatrix}
\begin{bmatrix} \frac{1}{\sqrt{2}} & \frac{1}{\sqrt{2}} & 0 \\ \frac{1}{\sqrt{18}} & -\frac{1}{\sqrt{18}} & \frac{4}{\sqrt{18}}\\ \frac{2}{3} & -\frac{2}{3} & -\frac{1}{3} \end{bmatrix}$

As I want to factorize $A$ I will be looking for $B$ and $C$, such that $A = BC$, where $B\in I\!R^{2xR}$ and $C \in I\!R^{Rx3}$

#### Now consider R=1
$A = 
\begin{bmatrix} \frac{1}{\sqrt{2}}   \\ \frac{1}{\sqrt{2}}  \end{bmatrix}
\begin{bmatrix} 5  \end{bmatrix}
\begin{bmatrix} \frac{1}{\sqrt{2}} & \frac{1}{\sqrt{2}} & 0  \end{bmatrix}$

  $ = 
\begin{bmatrix} \frac{5}{\sqrt{2}}   \\ \frac{5}{\sqrt{2}}  \end{bmatrix}
\begin{bmatrix} \frac{1}{\sqrt{2}} & \frac{1}{\sqrt{2}} & 0  \end{bmatrix}$

Finally, the number of parameters that we will need to store will be <font color=green>$(2+3) \times 1 = 5$</font> 

In [21]:
import numpy as np

In [54]:
A = np.array([[3,2,2],[3,2,-2]])
A

array([[ 3,  2,  2],
       [ 3,  2, -2]])

In [55]:
U, S, V_t = np.linalg.svd(A, full_matrices=True)
print('\n',A.shape,'\n',U.shape,'\n',S.shape,'\n',V_t.shape)


 (2, 3) 
 (2, 2) 
 (2,) 
 (3, 3)


In [56]:
U

array([[-0.70710678, -0.70710678],
       [-0.70710678,  0.70710678]])

In [57]:
S

array([5.09901951, 2.82842712])

In [58]:
V_t

array([[-8.32050294e-01, -5.54700196e-01, -2.22044605e-16],
       [ 5.55111512e-17, -5.55111512e-17, -1.00000000e+00],
       [-5.54700196e-01,  8.32050294e-01,  0.00000000e+00]])

### Lets take R=1

In [59]:
B = U[:,0] * S[0]
B

array([-3.60555128, -3.60555128])

In [60]:
C = V_t[0,:]
C

array([-8.32050294e-01, -5.54700196e-01, -2.22044605e-16])

In [61]:
A_ = np.outer(B,C)
A_

array([[3.00000000e+00, 2.00000000e+00, 8.00593208e-16],
       [3.00000000e+00, 2.00000000e+00, 8.00593208e-16]])

In [62]:
abs(A_ - A) < 0.000001

array([[ True,  True, False],
       [ True,  True, False]])

## Tensor decomposition 
##### CP decomposition
$T\in I\!R^{d1 \times d2 \times d3}$, we will decompose our tensor into $A\in I\!R^{d1 \times R}$ , $B\in I\!R^{d2 \times R}$ and $C\in I\!R^{d3 \times R}$
##### Tucker decomposition
$T\in I\!R^{d1 \times d2 \times d3}$, we will decompose our tensor into $G\in I\!R^{R1 \times R2 \times R3}$ and $U_i\in I\!R^{d_i \times R_i} $ for $i$ in [1,2,3]
+ $G$ is a core tensor
+ $U_i$ are factor matrices

##### Tensor Train decomposition
$T\in I\!R^{d1 \times d2 \times d3 \times d4}$, we will decompose our tensor into $G_1\in I\!R^{d1 \times R1}$ , $G_2\in I\!R^{R1 \times d2 \times R2}$ , $G_3\in I\!R^{R2 \times d3 \times R3}$ , $G_4\in I\!R^{R_4 \times d_4}$  
+ $G_i$ are a core tensors